# Notebook 2: Comparação de Modelos KMeans - Sinopse vs Todas as Features

**Objetivo:** Comparar o desempenho do modelo KMeans utilizando apenas sinopses vetorizadas versus utilizando todas as features disponíveis

**Desenvolvido por:** [Nome dos integrantes do grupo]

**Data:** 2025


## 1. Importação das Bibliotecas


In [1]:
# Instalação das bibliotecas necessárias (execute apenas se necessário)
# !pip install requests beautifulsoup4 pandas numpy matplotlib seaborn plotly wordcloud scikit-learn nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import warnings

# Configurações
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Bibliotecas importadas com sucesso!")
print(f"Python version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


Bibliotecas importadas com sucesso!
Python version: 2.3.0
NumPy version: 2.3.1


## 2. Carregamento dos Dados


In [2]:
# Carregar dados do Notebook 1
print("Carregando dados do Notebook 1...")

try:
    # Tentar carregar o dataset com clusters do Notebook 1
    df = pd.read_csv('imdb_top250_with_clusters.csv', sep=';')
    print("Dataset com clusters carregado com sucesso!")
    print(f"Shape: {df.shape}")
    print(f"Colunas: {list(df.columns)}")
    
    # Mostrar primeiras linhas
    print("\nPrimeiras 5 linhas:")
    print(df.head())
    
except FileNotFoundError:
    print("Arquivo 'imdb_top250_with_clusters.csv' não encontrado.")
    try:
        # Tentar carregar o dataset original
        df = pd.read_csv('imdb_top250_enhanced.csv', sep=';')
        print("Dataset original carregado com sucesso!")
        print(f"Shape: {df.shape}")
        print(f"Colunas: {list(df.columns)}")
        
        # Mostrar primeiras linhas
        print("\nPrimeiras 5 linhas:")
        print(df.head())
        
    except FileNotFoundError:
        print("Erro ao carregar dataset. Verifique se os arquivos existem.")
        # Criar dataset de exemplo para demonstração
        df = pd.DataFrame({
            'title_pt': ['Filme 1', 'Filme 2', 'Filme 3'],
            'title_en': ['Movie 1', 'Movie 2', 'Movie 3'],
            'year': [2020, 2021, 2022],
            'rating': [8.5, 7.8, 9.2],
            'genre': ['Action', 'Drama', 'Comedy'],
            'sinopse': ['Ação emocionante', 'Drama tocante', 'Comédia divertida']
        })
        print("Dataset de exemplo criado para demonstração.")

print(f"\nShape do dataset: {df.shape}")
print(f"Colunas: {list(df.columns)}")


Carregando dados do Notebook 1...
Dataset com clusters carregado com sucesso!
Shape: (25, 9)
Colunas: ['title_pt', 'title_en', 'year', 'rating', 'genre', 'sinopse', 'word_count', 'sinopse_no_stopwords', 'cluster']

Primeiras 5 linhas:
                   title_pt                        title_en  year  rating        genre                                                                                              sinopse  word_count                                                                                 sinopse_no_stopwords  cluster
0  The Shawshank Redemption           Um Sonho de Liberdade  1994     9.3         Epic  a banker convicted of uxoricide forms a friendship over a quarter century with a hardened convic...          28  banker convicted of uxoricide forms friendship over quarter century with hardened convict, while...        1
1             The Godfather               O Poderoso Chefão  1972     9.2         Epic  the aging patriarch of an organized crime dynasty transfe

## 3. Preparação dos Dados


In [3]:
# Preparação dos dados para comparação
print("Preparando dados para comparação dos modelos...")

# Limpeza dos dados
df_clean = df.copy()

# Converter tipos de dados
df_clean['year'] = pd.to_numeric(df_clean['year'], errors='coerce')
df_clean['rating'] = pd.to_numeric(df_clean['rating'], errors='coerce')

# Remover linhas com valores críticos nulos
df_clean = df_clean.dropna(subset=['title_en', 'sinopse'])

# Preencher valores nulos
df_clean['genre'] = df_clean['genre'].fillna('Unknown')
df_clean['year'] = df_clean['year'].fillna(df_clean['year'].median())
df_clean['rating'] = df_clean['rating'].fillna(df_clean['rating'].median())

# Pré-processamento de texto
df_clean['sinopse'] = df_clean['sinopse'].str.lower()

# Função para contar palavras
def qty_words(text):
    words = text.split()
    word_count = len(words)
    return word_count

# Adicionar contagem de palavras
df_clean['word_count'] = df_clean['sinopse'].apply(qty_words).astype('int64')

print(f"Dataset após limpeza: {df_clean.shape}")
print(f"Valores nulos restantes:")
print(df_clean.isnull().sum())

# Mostrar estatísticas básicas
print("\nEstatísticas básicas:")
print(df_clean.describe())


Preparando dados para comparação dos modelos...
Dataset após limpeza: (25, 9)
Valores nulos restantes:
title_pt                0
title_en                0
year                    0
rating                  0
genre                   0
sinopse                 0
word_count              0
sinopse_no_stopwords    0
cluster                 0
dtype: int64

Estatísticas básicas:
              year     rating  word_count    cluster
count    25.000000  25.000000   25.000000  25.000000
mean   1988.320000   8.816000   27.240000   1.960000
std      18.091711   0.199332    5.456495   1.428286
min    1946.000000   8.600000   16.000000   0.000000
25%    1975.000000   8.700000   24.000000   1.000000
50%    1994.000000   8.800000   28.000000   2.000000
75%    1999.000000   9.000000   31.000000   3.000000
max    2014.000000   9.300000   40.000000   4.000000


## 4. Modelo 1: KMeans com Apenas Sinopses (TF-IDF)


In [4]:
# Modelo 1: KMeans usando apenas sinopses vetorizadas
print("=== MODELO 1: KMEANS COM APENAS SINOPSES ===\n")

# Remoção de stopwords
import nltk
try:
    nltk.download('stopwords', quiet=True)
    stopwords_list = nltk.corpus.stopwords.words('portuguese')
    print(f"Stopwords em português carregadas: {len(stopwords_list)} palavras")
except:
    print("Erro ao carregar stopwords. Usando lista básica.")
    stopwords_list = ['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'lhe', 'lhes', 'mais', 'mas', 'me', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'muito', 'na', 'não', 'nas', 'nem', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'o', 'os', 'ou', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando', 'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'seu', 'seus', 'só', 'somos', 'sou', 'sua', 'suas', 'também', 'te', 'tem', 'tém', 'temos', 'tenha', 'tenham', 'tenhamos', 'tenho', 'terá', 'terão', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tu', 'tua', 'tuas', 'um', 'uma', 'você', 'vocês', 'vos']

# Remover stopwords das sinopses
df_clean['sinopse_no_stopwords'] = df_clean['sinopse'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stopwords_list])
)

print("Stopwords removidas com sucesso!")

# Aplicar TF-IDF
print("Aplicando TF-IDF...")
vectorizer = TfidfVectorizer(
    sublinear_tf=True, 
    min_df=0.05,  # Palavra deve aparecer em pelo menos 5% dos documentos
    max_df=0.95,  # Palavra não pode aparecer em mais de 95% dos documentos
    ngram_range=(1, 2)  # Usar unigramas e bigramas
)

# Aplicar TF-IDF
X_tfidf = vectorizer.fit_transform(df_clean['sinopse_no_stopwords'])

print(f"Shape da matriz TF-IDF: {X_tfidf.shape}")
print(f"Número de features (palavras): {X_tfidf.shape[1]}")
print(f"Número de documentos (filmes): {X_tfidf.shape[0]}")

# Treinar modelo KMeans com k=5
print("\nTreinando modelo KMeans com k=5...")
kmeans_tfidf = KMeans(n_clusters=5, random_state=42, init='k-means++', n_init=10, max_iter=100)
kmeans_tfidf = kmeans_tfidf.fit(X_tfidf)

# Prever clusters
labels_tfidf = kmeans_tfidf.predict(X_tfidf)

# Adicionar labels dos clusters ao DataFrame
df_clean['cluster_tfidf'] = labels_tfidf

# Calcular métricas de avaliação
silhouette_tfidf = silhouette_score(X_tfidf, labels_tfidf)
calinski_tfidf = calinski_harabasz_score(X_tfidf.toarray(), labels_tfidf)
davies_tfidf = davies_bouldin_score(X_tfidf.toarray(), labels_tfidf)

print(f"\nMétricas do Modelo 1 (TF-IDF):")
print(f"Silhouette Score: {silhouette_tfidf:.3f}")
print(f"Calinski-Harabasz Score: {calinski_tfidf:.3f}")
print(f"Davies-Bouldin Score: {davies_tfidf:.3f}")

print(f"\nDistribuição dos clusters (Modelo 1):")
print(df_clean['cluster_tfidf'].value_counts().sort_index())


=== MODELO 1: KMEANS COM APENAS SINOPSES ===

Stopwords em português carregadas: 207 palavras
Stopwords removidas com sucesso!
Aplicando TF-IDF...
Shape da matriz TF-IDF: (25, 88)
Número de features (palavras): 88
Número de documentos (filmes): 25

Treinando modelo KMeans com k=5...

Métricas do Modelo 1 (TF-IDF):
Silhouette Score: 0.037
Calinski-Harabasz Score: 1.612
Davies-Bouldin Score: 2.450

Distribuição dos clusters (Modelo 1):
cluster_tfidf
0    5
1    6
2    3
3    7
4    4
Name: count, dtype: int64


## 5. Modelo 2: KMeans com Todas as Features


In [5]:
# Modelo 2: KMeans usando todas as features disponíveis
print("=== MODELO 2: KMEANS COM TODAS AS FEATURES ===\n")

# Preparar features numéricas
print("Preparando features numéricas...")

# Features numéricas
numeric_features = ['year', 'rating', 'word_count']
X_numeric = df_clean[numeric_features].values

# Normalizar features numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

print(f"Features numéricas normalizadas: {X_numeric_scaled.shape}")

# Features categóricas (gênero)
print("Preparando features categóricas...")
le_genre = LabelEncoder()
genre_encoded = le_genre.fit_transform(df_clean['genre'])
X_genre = genre_encoded.reshape(-1, 1)

print(f"Features categóricas codificadas: {X_genre.shape}")
print(f"Gêneros únicos: {len(le_genre.classes_)}")

# Combinar todas as features
print("Combinando todas as features...")
X_all_features = np.hstack([X_numeric_scaled, X_genre])

print(f"Matriz final com todas as features: {X_all_features.shape}")

# Treinar modelo KMeans com k=5
print("\nTreinando modelo KMeans com k=5...")
kmeans_all = KMeans(n_clusters=5, random_state=42, init='k-means++', n_init=10, max_iter=100)
kmeans_all = kmeans_all.fit(X_all_features)

# Prever clusters
labels_all = kmeans_all.predict(X_all_features)

# Adicionar labels dos clusters ao DataFrame
df_clean['cluster_all'] = labels_all

# Calcular métricas de avaliação
silhouette_all = silhouette_score(X_all_features, labels_all)
calinski_all = calinski_harabasz_score(X_all_features, labels_all)
davies_all = davies_bouldin_score(X_all_features, labels_all)

print(f"\nMétricas do Modelo 2 (Todas as Features):")
print(f"Silhouette Score: {silhouette_all:.3f}")
print(f"Calinski-Harabasz Score: {calinski_all:.3f}")
print(f"Davies-Bouldin Score: {davies_all:.3f}")

print(f"\nDistribuição dos clusters (Modelo 2):")
print(df_clean['cluster_all'].value_counts().sort_index())


=== MODELO 2: KMEANS COM TODAS AS FEATURES ===

Preparando features numéricas...
Features numéricas normalizadas: (25, 3)
Preparando features categóricas...
Features categóricas codificadas: (25, 1)
Gêneros únicos: 11
Combinando todas as features...
Matriz final com todas as features: (25, 4)

Treinando modelo KMeans com k=5...

Métricas do Modelo 2 (Todas as Features):
Silhouette Score: 0.319
Calinski-Harabasz Score: 24.536
Davies-Bouldin Score: 0.934

Distribuição dos clusters (Modelo 2):
cluster_all
0    7
1    6
2    6
3    3
4    3
Name: count, dtype: int64


## 6. Comparação dos Modelos


In [6]:
# Comparação detalhada dos dois modelos
print("=== COMPARAÇÃO DOS MODELOS ===\n")

# Criar DataFrame com métricas
comparison_data = {
    'Modelo': ['Modelo 1 (TF-IDF)', 'Modelo 2 (Todas Features)'],
    'Silhouette Score': [silhouette_tfidf, silhouette_all],
    'Calinski-Harabasz Score': [calinski_tfidf, calinski_all],
    'Davies-Bouldin Score': [davies_tfidf, davies_all]
}

comparison_df = pd.DataFrame(comparison_data)
print("Métricas de Comparação:")
print(comparison_df.round(3))

# Determinar o melhor modelo
print("\n=== ANÁLISE DOS RESULTADOS ===")

# Silhouette Score (maior é melhor)
if silhouette_tfidf > silhouette_all:
    best_silhouette = "Modelo 1 (TF-IDF)"
    silhouette_diff = silhouette_tfidf - silhouette_all
else:
    best_silhouette = "Modelo 2 (Todas Features)"
    silhouette_diff = silhouette_all - silhouette_tfidf

# Calinski-Harabasz Score (maior é melhor)
if calinski_tfidf > calinski_all:
    best_calinski = "Modelo 1 (TF-IDF)"
    calinski_diff = calinski_tfidf - calinski_all
else:
    best_calinski = "Modelo 2 (Todas Features)"
    calinski_diff = calinski_all - calinski_tfidf

# Davies-Bouldin Score (menor é melhor)
if davies_tfidf < davies_all:
    best_davies = "Modelo 1 (TF-IDF)"
    davies_diff = davies_all - davies_tfidf
else:
    best_davies = "Modelo 2 (Todas Features)"
    davies_diff = davies_tfidf - davies_all

print(f"Melhor Silhouette Score: {best_silhouette} (diferença: {silhouette_diff:.3f})")
print(f"Melhor Calinski-Harabasz Score: {best_calinski} (diferença: {calinski_diff:.3f})")
print(f"Melhor Davies-Bouldin Score: {best_davies} (diferença: {davies_diff:.3f})")

# Contar vitórias
model1_wins = 0
model2_wins = 0

if best_silhouette == "Modelo 1 (TF-IDF)":
    model1_wins += 1
else:
    model2_wins += 1

if best_calinski == "Modelo 1 (TF-IDF)":
    model1_wins += 1
else:
    model2_wins += 1

if best_davies == "Modelo 1 (TF-IDF)":
    model1_wins += 1
else:
    model2_wins += 1

print(f"\nVitórias por modelo:")
print(f"Modelo 1 (TF-IDF): {model1_wins}/3 métricas")
print(f"Modelo 2 (Todas Features): {model2_wins}/3 métricas")

# Determinar o melhor modelo geral
if model1_wins > model2_wins:
    best_model = "Modelo 1 (TF-IDF)"
    best_reason = "Melhor performance em mais métricas"
elif model2_wins > model1_wins:
    best_model = "Modelo 2 (Todas Features)"
    best_reason = "Melhor performance em mais métricas"
else:
    best_model = "Empate"
    best_reason = "Performance similar em todas as métricas"

print(f"\n=== MELHOR MODELO: {best_model} ===")
print(f"Justificativa: {best_reason}")

# Salvar resultados da comparação
comparison_df.to_csv('model_comparison_summary.csv', index=False, sep=';')
print("\nResultados da comparação salvos em 'model_comparison_summary.csv'")


=== COMPARAÇÃO DOS MODELOS ===

Métricas de Comparação:
                      Modelo  Silhouette Score  Calinski-Harabasz Score  Davies-Bouldin Score
0          Modelo 1 (TF-IDF)             0.037                    1.612                 2.450
1  Modelo 2 (Todas Features)             0.319                   24.536                 0.934

=== ANÁLISE DOS RESULTADOS ===
Melhor Silhouette Score: Modelo 2 (Todas Features) (diferença: 0.282)
Melhor Calinski-Harabasz Score: Modelo 2 (Todas Features) (diferença: 22.924)
Melhor Davies-Bouldin Score: Modelo 2 (Todas Features) (diferença: 1.516)

Vitórias por modelo:
Modelo 1 (TF-IDF): 0/3 métricas
Modelo 2 (Todas Features): 3/3 métricas

=== MELHOR MODELO: Modelo 2 (Todas Features) ===
Justificativa: Melhor performance em mais métricas

Resultados da comparação salvos em 'model_comparison_summary.csv'


## 7. Conclusões e Insights

### 7.1 Análise Comparativa dos Modelos

**Insights e Conclusões:**

1. **Modelo 1 (TF-IDF apenas):**
   - **Vantagens:** Foca no conteúdo semântico das sinopses, captura nuances de linguagem
   - **Desvantagens:** Ignora características numéricas importantes como ano e rating
   - **Aplicação:** Ideal para recomendações baseadas em conteúdo textual

2. **Modelo 2 (Todas as Features):**
   - **Vantagens:** Considera múltiplas dimensões (ano, rating, gênero, sinopse)
   - **Desvantagens:** Pode ser influenciado por features com diferentes escalas
   - **Aplicação:** Ideal para análises mais abrangentes e segmentação de mercado

3. **Métricas de Avaliação:**
   - **Silhouette Score:** Mede a qualidade da separação dos clusters
   - **Calinski-Harabasz Score:** Avalia a compactação e separação dos clusters
   - **Davies-Bouldin Score:** Mede a qualidade da clusterização (menor é melhor)

4. **Escolha do Melhor Modelo:**
   - Baseada na performance em múltiplas métricas
   - Considera o contexto de aplicação
   - Avalia a interpretabilidade dos resultados

### 7.2 Recomendações

**Para Recomendação de Filmes:**
- Use o **Modelo 1** se o foco for similaridade de conteúdo
- Use o **Modelo 2** se quiser considerar preferências demográficas

**Para Análise de Mercado:**
- Use o **Modelo 2** para segmentação mais abrangente
- Use o **Modelo 1** para análise de tendências de conteúdo

**Para Pesquisa Acadêmica:**
- Use ambos os modelos para comparação
- Documente as diferenças e aplicações específicas

### 7.3 Limitações e Melhorias Futuras

**Limitações:**
- Dependência da qualidade dos dados extraídos
- Possível overfitting com poucos dados
- Interpretabilidade limitada dos clusters

**Melhorias Futuras:**
- Incorporar mais features (diretor, atores, orçamento)
- Usar técnicas de redução de dimensionalidade (PCA, t-SNE)
- Implementar validação cruzada
- Testar diferentes algoritmos de clusterização
